# Répertoire des bilans sociaux de grandes entreprises - focus sur les salariés en situation de handicap

**Objectif** : extraction et mise en forme des données de bilan social d'entreprises similaires à EDF liées aux salariés en situation de handicap 

**Méthodologie** : 
- exploration des données d'EDF et de leur format
- établissement d'une liste d'entreprises cibles
- téléchargement des Déclarations de Performance Extra-Financière (DPEF) des entreprises cibles
- extraction de tous les tableaux de données des DPEF (csv)
- extration des informations liées aux salariés en situation de handicap des tableaux de données
- mise en forme des informations pour correspondre à celles de EDF

**Outils et librairies associées**:
- extraction des tableaux des fichiers pdf avec *docling* (https://github.com/DS4SD/docling)
- extraction des informations ciblées avec le framework *langchain* (https://www.langchain.com/)
- modèle de langage *Ollama Qwen2.5* en local pour la sélection des informations (https://ollama.com/library/qwen2.5:7b)

## Liste entreprises extérieures et documents disponibles

Entreprises ciblées : CNP Assurances, ENGIE, Décathlon, Carrefour, Auchan, Orange.

La liste des entreprises ciblées comporte celles données dans les ressources du défi ainsi que d'autres : les plus importantes en France ou celles d'effectif similaire à EDF, en 2018 (https://fr.wikipedia.org/wiki/Liste_des_plus_grandes_entreprises_fran%C3%A7aises). Les données de l'INSA n'ont pas été extraites car le document ne contient pas de tableau pour les effectifs des salariés en situation de handicap.

Au vu de la nomenclature unique de chaque fichier des différentes entreprises, l'absence de données brutes, la présence de données pour l'année en cours ou précédente seulement, la méthode d'extraction des données ne semble pas pouvoir être totalement automatisée. 

### Documents disponibles

**Bilan sociaux**

|Entreprise|Type données|Lien|Commentaire |
|----------|------------|----|------------|
|CNP Assurances |**pdf texte**| https://www.cnp.fr/cnp/content/download/11474/file/CNP-Assurances-Bilan-social-2023.pdf | Information handicap présente |
|ENGIE          |**pdf texte**|https://www.engie.com/sites/default/files/assets/documents/2023-03/ENGIE%20SA_Bilan%20social%202021_VD.pdf | Information handicap présente |
|INSA Strasbourg|**pdf**|https://www.insa-strasbourg.fr/wp-content/uploads/INSA_bilan_soc_20_V2_21.pdf | Information handicap non présente |

**Déclaration de Performance Extra-Financière (DPEF)**

|Entreprise|Type données|Lien|Commentaire|
|----------|------------|----|-----------|
|Décathlon|**html vers pdfs texte**| https://engagements.decathlon.fr/les-rapports-developpement-durable-decathlon-annuels | tableaux en annexe dispo par pays - p92 pour handicap|
|Carrefour|**pdf texte**|https://www.carrefour.com/sites/default/files/2024-05/DPEF%202023%20Groupe%20Carrefour.pdf| tableau évolution 2022-23 p58|
|Carrefour|**pdf texte**|https://www.carrefour.com/sites/default/files/2023-04/DPEF_Carrefour_2022.pdf| pour 2021-22 p131|
|Auchan|**html vers pdf texte**|https://www.auchan-retail.com/fr/rapport-financier-annuel-et-declaration-de-performance-extra-financiere-2022/| tableau évolution 2020-2023 p98|
|Orange|**pdf texte**|https://gallery.orange.com/rse/?v=ffca4aaa-5c3b-44e4-ba7b-2760163650ea#beecontext=viewShareContext&l=row&st=417c9e1e-3eae-44b2-bce6-c1a6dadc179f|il faut regarder dans chaque document|

**Autres documents ou données**

|Entreprise|Type données|Lien|Commentaire |
|----------|------------|----|------------|
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/agents-situation-handicap/table/?sort=date | Spécifique handicap / an |
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/nombre-total-agents-effectifs/table/?sort=date | Spécifique par collège / an |
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/repartition-genre-effectif/information/ | Spécifique par genre / an |


# Extraction des informations sur les salariés handicapés de plusieurs entreprises

Ci-dessous, le pipeline complet des fichiers pdfs à un fichier csv avec les informations ciblées (nombre ou pourcentage de salariés en situation de handicap en France, pour plusieurs années).

## Librairies et configuration

In [1]:
import glob
import re 

from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

import pandas as pd 
import numpy as np

from langchain_ollama import OllamaLLM

## Fonctions 

In [2]:
def pdf_tables_to_csv(liste_fichiers_pdf):
    """Extraction des tableaux des fichiers pdf donnés dans la liste en plusieurs fichiers csv    
    """
    # Paramètres pour la conversion des fichiers pdf
    pipeline_options = PdfPipelineOptions()
    pipeline_options.do_ocr = False
    pipeline_options.do_table_structure = True
    pipeline_options.table_structure_options.do_cell_matching = False

    for input_doc_path in liste_fichiers_pdf:
        # Instanciation du convertisseur de pdf
        doc_converter = DocumentConverter(
            format_options={
                InputFormat.PDF: PdfFormatOption(
                    pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend)
                            }
        )

        # Conversion du pdf
        conv_result = doc_converter.convert(input_doc_path)

        # Export des tableaux 
        for table_ix, table in enumerate(conv_result.document.tables):
            table_df: pd.DataFrame = table.export_to_dataframe()
            # Sauvegarde en csv
            element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
            table_df.to_csv(element_csv_filename)

In [3]:
def extract_handicap_informations(entreprise, limit=-1) -> dict:
    """ Recherche parmis tous les fichiers csv d'une entreprise les informations et données 
    relatives aux salariés en situation de handicap
    
    Input : nom de l'entreprise (doit correspondre à un nom de dossier dans ../data/raw)

    Output : dictionnaire {'nom_du_fichier.csv' : 'sortie du llm'}
    """

    fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

    resultats = {}

    if limit==-1:
        limit=len(fichiers_entreprise)
    for fichier in fichiers_entreprise[:limit]:
        data = pd.read_csv(fichier)

        # si le mot 'handicap' est présent dans le tableau, on recherche les informations, sinon on retourne un dictionnaire sans rien
        if len(re.findall(r".*handicap.*", data.to_string()))!=0:

            model = OllamaLLM(model="qwen2.5", temperature=0)

            template = '''
            Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
            Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
            1. Nombre total de salariés en situation de handicap en France ou en Europe.
            2. Pourcentage ou taux de salariés total en situation de handicap en France ou en Europe.
            3. Le pays ou la région associés (France, Europe ou autre).
            Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.
            Si tu ne connais pas l'année, retourne `null` pour sa valeur.
            Tu ne fais aucun calcul, ne suppose rien, et ré-écris simplement les valeurs trouvées dans les données initiales.

            ### DONNES INITIALES :

            {data}


            ### LISTE FINALE (EXEMPLE) :

            [
                "année : <valeur> et pays <valeur région ou pays>, salariés en situation de handicap : <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
                ...
            ]


            ### RÉPONSE :

            '''.format(data = data, entreprise=entreprise)

            result = model.invoke(template)
        else:
            result = 'pas de mot handicap dans le fichier'
        resultats[f'{fichier}'] = result
        # print(f"Réponse du LLM pour le fichier {fichier}\n", result, '\n')

    return resultats



In [4]:

def parse_results_to_df(liste_output_llm) -> pd.DataFrame:
    """Transforme la chaîne de caractères en sortie de prompt+modèle en df
    
    Input : liste de chaînes de caractères du type :
        [
            'année : 2020, salariés en situation de handicap : null, pourcentage handicap : 4,0 %',
            'année : 2021, salariés en situation de handicap : null, pourcentage handicap : 4,6 %',
            'année : 2022, salariés en situation de handicap : null, pourcentage handicap : 4,1 %'
        ]

    """
    # instanciation du df des résultats du tableau i sous la forme de liste
    df = pd.DataFrame({  
                    'année':[''],
                    'salariés en situation de handicap':[''],
                    'pourcentage handicap':['']
                    })

    if liste_output_llm is not None : #si liste_output_llm est None alors le df output contient des caractères vides
        for result_string in liste_output_llm:
            # séparation des attribut et valeurs associées
            result_string_splitted = re.split(', ', result_string)
            # instanciation des listes d'attributs pour l'année en cours
            annee_liste = []
            nombre_salaries_handicap = []
            pourcentage_handicap = []
            for string in result_string_splitted:
                # séparation du nom de l'attribut de sa valeur
                key, value = re.split(":", string)
                # ajout des valeurs aux listes selon les noms d'attribut
                if key.strip() == "année":
                    annee_liste.append(value.strip())
                elif key.strip() =="salariés en situation de handicap":
                    nombre_salaries_handicap.append(value.strip())
                elif key.strip()=="pourcentage handicap":
                    pourcentage_handicap.append(value.strip())
                else:
                    print("Error : stopping.")
            # construction d'un tableau intermédiaire pour l'année en cours
            df_tmp = pd.DataFrame({  
                            'année':annee_liste,
                            'salariés en situation de handicap':nombre_salaries_handicap,
                            'pourcentage handicap':pourcentage_handicap
                            })
            # merge avec le dataframe final
            df = pd.concat([df, df_tmp])
        #suppression de la première ligne vide
        df = df.iloc[1:]
    
    return df

In [5]:
def parse_llms_results_to_df(results, entreprise_name):
    """ Transforme le dictionnaire contenant les outputs du llm en df
    
    Input : Results = Dictionnaire { 'nom_fichier' : 'output llm'}
            entreprise_name = String nom de l'entreprise en cours 
    """

    df_all = None
    for key in results.keys():
        text_raw = results[f'{key}']
        text_clean = re.sub('\n', '', text_raw)
        text_clean = re.sub('pourcentage handicap : null', 'pourcentage handicap : null %', text_clean)
        text_search = re.search(r'.*(\[    "année : .*%"\]).*', text_clean)
        if text_search is not None : 
            text_extracted=text_search.group(1)
            liste_texte = eval(text_extracted)
        else: 
            liste_texte = None
        
        df_tmp = parse_results_to_df(liste_texte)
        
        df_tmp['source']=f'{key}'
        df_tmp['entreprise']=f'{entreprise_name}'
        if df_all is None:
            df_all=df_tmp
        else:
            df_all = pd.concat([df_all, df_tmp])
    return df_all


In [6]:
def extract_handicap_information_all_entreprises(liste_entreprise_directories)->pd.DataFrame:
    """ 
        Création du dataframe avec les informations des salariés handicapés à partir des noms des entreprises

        Input : liste des noms d'entreprises correspondant aux noms des dossiers dans data/raw 

        Output : df brut des données extraites des fichiers csv
    """
    df_all_entreprise=None
    for entreprise in liste_entreprise_directories:
        assert len(glob.glob(f'../data/raw/{entreprise}'))==1, "Erreur : le nom de l'entreprise ne correspond pas à un dossier unique dans data/raw."
        resultats = extract_handicap_informations(entreprise, limit=-1)
        df_tmp = parse_llms_results_to_df(results=resultats, entreprise_name=entreprise)
        if df_all_entreprise is None:
            df_all_entreprise=df_tmp
        else:
            df_all_entreprise=pd.concat([df_all_entreprise, df_tmp])
    return df_all_entreprise

In [7]:
def clean_handicap_information_all_entreprises(df_raw):
    """ 
    Nettoyage du dataframe de sortie pour correspondre au format fourni dans le fichier d'EDF
    """
    mask = (
            #sélection des lignes avec des années écrites
            (df_raw.année.str.contains(r'\d', regex=True)) 
            #sélection des lignes avec un nb de salariés ou un %
            &(
                df_raw['salariés en situation de handicap'].str.contains(r'\d', regex=True) 
                |
                df_raw['pourcentage handicap'].str.contains(r'\d', regex=True)
                ) 
            )
    df_tmp = df_raw.loc[mask]

    # Séparation de l'année et du pays
    df_annee_pays = df_tmp.année.str.split(' et pays ', expand=True).set_axis(['année', 'pays'], axis=1)
    df_tmp['année']=df_annee_pays.loc[:,'année'].values
    df_tmp['pays']=df_annee_pays.loc[:,'pays'].values

    # Sélection des données pour France ou null si France non mentionné
    mask = df_tmp.pays.str.contains('France')|df_tmp.pays.str.contains('null')
    df_tmp = df_tmp.loc[mask]

    # Extraction des pourcentages en valeur transformable en float
    pourcentages_clean = [re.sub(r'.*(\d+).(\d+).*', r'\1.\2', value) for value in df_tmp['pourcentage handicap'].values]
    pourcentages_clean = [re.sub('%', '', pourcentage).strip() 
                            if re.match(r'.*%', pourcentage) is not None 
                            else pourcentage 
                            for pourcentage in pourcentages_clean]
    df_tmp['pourcentage handicap float'] = pourcentages_clean
    # Transformation en float
    df_tmp['pourcentage handicap float'] = df_tmp['pourcentage handicap float'].astype(float)

    # Transformation des salariés en situation de handicap en float 
    nombre_clean = [re.sub(r'\W', '', value) for value in df_tmp['salariés en situation de handicap'].values]
    df_tmp['nombre'] = nombre_clean
    df_tmp = df_tmp.replace(to_replace={'null':np.nan})

    # Pour le nombre - Sélection des variables et renommage des colonnes
    df_final_nombre = (df_tmp
                    .loc[:, ['année', 'entreprise', 'nombre', 'pays']]
                    .rename(columns = {'année':'Année',
                                        "pays":"Perimètre spatial",
                                        'entreprise':'Perimètre juridique',
                                        'nombre':'Valeur',
                                        
                                        })
                )
    df_final_nombre['Indicateur'] = 'Salariés en situation de handicap'
    df_final_nombre['Unité'] = 'nombre'

    # Pour le pourcentage 
    df_final_pourcentage = (df_tmp
                    .loc[:, ['année', 'entreprise', 'pourcentage handicap float', 'pays']]
                    .rename(columns = {'année':'Année',
                                        "pays":"Perimètre spatial",
                                        'entreprise':'Perimètre juridique',
                                        'pourcentage handicap float':'Valeur',
                                        
                                        })
                )
    df_final_pourcentage['Indicateur'] = 'Salariés en situation de handicap (%)'
    df_final_pourcentage['Unité'] = 'pourcentage'

    # Concaténation du nombre et du pourcentage
    df_final = pd.concat([df_final_nombre, df_final_pourcentage], axis=0)
    
    return df_final

## Application aux fichiers des entreprises Auchan, Carrefour, CNP, Decathlon, ENGIE et Orange

Application uniquement aux fichiers pdf dont quelques pages ont été extraites (dont celle avec l'information ciblée), pour gagner du temps de calcul.

In [8]:
# Input : liste des fichiers pdf à convertir (fichiers minimaux avec les pages d'intérêt sélectionnées au préalable)
liste_fichiers_pdf = [path for path in glob.glob('../data/raw/*/*') if re.match(r'.*_\dp\.pdf', path)]
print(liste_fichiers_pdf)

# Noms des entreprises avec des fichiers pdf associés
liste_entreprises = list(np.unique([re.sub(r'.*/raw/(.*)/.*', r'\1', fichier) for fichier in liste_fichiers_pdf]))
print(liste_entreprises)

# Génération des fichiers contentant les tableaux (csv)
pdf_tables_to_csv(liste_fichiers_pdf)

# Extraction des informations des fichiers csv
entreprises_raw = extract_handicap_information_all_entreprises(liste_entreprises)

# Nettoyage du dataframe et création du fichier csv de sortie 
entreprises_clean = clean_handicap_information_all_entreprises(entreprises_raw)

# Création du fichier de sortie 
output_path = "../data/processed/salaries-en-situation-handicap_entreprises-cibles.csv"
entreprises_clean.to_csv(output_path, index=False)


['../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p.pdf', '../data/raw/Carrefour/DPEF_Carrefour_2022_3p.pdf', '../data/raw/Orange/Bilan Social 2021 Orange SA_6p.pdf', '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p.pdf', '../data/raw/Auchan/auchan_2022_6p.pdf', '../data/raw/Decathlon/2023_Decathlon_Déclaration_de_Performance_Extra_Financière_2023_7p.pdf', '../data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021_9p.pdf', '../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p.pdf']
['Auchan', 'CNP', 'Carrefour', 'Decathlon', 'ENGIE', 'Orange']


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

/tmp/ipykernel_4865/834607274.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['année']=df_annee_pays.loc[:,'année'].values
/tmp/ipykernel_4865/834607274.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['pays']=df_annee_pays.loc[:,'pays'].values


# Comparaison des données extraites et des données réelles

**Données réelles** 

|Année|Indicateur|Valeur|Perimètre juridique|Unité|
|------|---------|-------|------------------|-----|
|2020|Salariés en situation de handicap||Auchan|nombre|
|2021|Salariés en situation de handicap||Auchan|nombre|
|2022|Salariés en situation de handicap||Auchan|nombre|
|2021|Salariés en situation de handicap|10902|Carrefour|nombre|
|2022|Salariés en situation de handicap|11281|Carrefour|nombre|
|2023|Salariés en situation de handicap|13358|Carrefour|nombre|
|2021|Salariés en situation de handicap|222|CNP|nombre|
|2022|Salariés en situation de handicap|231|CNP|nombre|
|2023|Salariés en situation de handicap|254|CNP|nombre|
|2020|Salariés en situation de handicap||Decathlon|nombre|
|2021|Salariés en situation de handicap||Decathlon|nombre|
|2022|Salariés en situation de handicap||Decathlon|nombre|
|2023|Salariés en situation de handicap||Decathlon|nombre|
|2019|Salariés en situation de handicap|189|ENGIE|nombre|
|2020|Salariés en situation de handicap|155|ENGIE|nombre|
|2021|Salariés en situation de handicap|154|ENGIE|nombre|
|2019|Salariés en situation de handicap|5247|Orange|nombre|
|2020|Salariés en situation de handicap|4748|Orange|nombre|
|2021|Salariés en situation de handicap|3685|Orange|nombre|
|2020|Salariés en situation de handicap (%)|4.0|Auchan|pourcentage|
|2021|Salariés en situation de handicap (%)|4.6|Auchan|pourcentage|
|2022|Salariés en situation de handicap (%)|4.1|Auchan|pourcentage|
|2021|Salariés en situation de handicap (%)|3.4|Carrefour|pourcentage|
|2022|Salariés en situation de handicap (%)|3.7|Carrefour|pourcentage|
|2023|Salariés en situation de handicap (%)|4.0|Carrefour|pourcentage|
|2021|Salariés en situation de handicap (%)|7.3|CNP|pourcentage|
|2022|Salariés en situation de handicap (%)|7.4|CNP|pourcentage|
|2023|Salariés en situation de handicap (%)|7.9|CNP|pourcentage|
|2020|Salariés en situation de handicap (%)|3.2|Decathlon|pourcentage|
|2021|Salariés en situation de handicap (%)|3.1|Decathlon|pourcentage|
|2022|Salariés en situation de handicap (%)|3.2|Decathlon|pourcentage|
|2023|Salariés en situation de handicap (%)|3.5|Decathlon|pourcentage|
|2019|Salariés en situation de handicap (%)||ENGIE|pourcentage|
|2020|Salariés en situation de handicap (%)||ENGIE|pourcentage|
|2021|Salariés en situation de handicap (%)||ENGIE|pourcentage|
|2019|Salariés en situation de handicap (%)||Orange|pourcentage|
|2020|Salariés en situation de handicap (%)||Orange|pourcentage|
|2021|Salariés en situation de handicap (%)||Orange|pourcentage|

**Données extraites**

|Année|Perimètre juridique|Valeur|Perimètre spatial|Indicateur|Unité|
|-----|-------------------|------|-----------------|----------|-----|
|2020|Auchan||France|Salariés en situation de handicap|nombre|
|2021|Auchan||France|Salariés en situation de handicap|nombre|
|2022|Auchan||France|Salariés en situation de handicap|nombre|
|2022|Carrefour|11281|France|Salariés en situation de handicap|nombre|
|2021|Carrefour|10902|France|Salariés en situation de handicap|nombre|
|2021|Decathlon||France|Salariés en situation de handicap|nombre|
|2022|Decathlon||France|Salariés en situation de handicap|nombre|
|2020|Decathlon||France|Salariés en situation de handicap|nombre|
|2021|Decathlon||France|Salariés en situation de handicap|nombre|
|2020|Auchan|4.0|France|Salariés en situation de handicap (%)|pourcentage|
|2021|Auchan|4.6|France|Salariés en situation de handicap (%)|pourcentage|
|2022|Auchan|4.1|France|Salariés en situation de handicap (%)|pourcentage|
|2022|Carrefour|3.7|France|Salariés en situation de handicap (%)|pourcentage|
|2021|Carrefour|3.4|France|Salariés en situation de handicap (%)|pourcentage|
|2021|Decathlon|3.1|France|Salariés en situation de handicap (%)|pourcentage|
|2022|Decathlon|3.2|France|Salariés en situation de handicap (%)|pourcentage|
|2020|Decathlon|3.2|France|Salariés en situation de handicap (%)|pourcentage|
|2021|Decathlon|3.1|France|Salariés en situation de handicap (%)|pourcentage|


**Comparaison des données obtenues et réelles**

- données correctes pour Auchan et Carrefour
- pas de données pour CNP (tableau à double entrée avec des sous-catégories et plusieurs lignes de totaux)
- données manquantes pour Decathlon : pour 2023 l'année n'a pas été extraite du pdf (non présente dans le csv)
- pas de donnée pour ENGIE et Orange ('handicap' ne figure pas dans le tableau)

# Remarques et perspectives d'amélioration

## Points facilitant l'extraction des données
- les entreprises ciblées utilisent toutes soit le format pdf (avec du texte), soit une possibilité de format csv

## Verrous
- extraction des informations des tableaux parfois incomplète (*ex : Décathlon 2023, pas d'année pour les données des salariés handicapés*)
- informations mises en page de manière différente selon les entreprises, les années, le type d'information 
- format des données variable (pourcentage, nombre, ratio, taux...) 
- tableau avec données par sous-catégories (pays, genre, autre) non extraites par le prompt (*ex : données de CNP*)
- valeurs inscrites dans les tableaux ne contiennent pas le mot "handicap" permettant de filtrer les tableaux à envoyer au LLM (*ex : données d'ENGIE et d'Orange*)

## Perspectives d'amélioration 
- extraire en markdown les fichiers pdf au lieu d'extraire uniquement les tableaux en csv 
- extraire et formatter avec docling les fichiers csv natifs (EDF, SNCF) pour améliorer l'adéquation avec langchain et le LLM
- cibler les pages contenant potentiellement l'information en amont (*ex : 5 pages autour d'une page contenant le mot clef "handicap"*)
- tester d'autres modèles de langage
- optimiser le prompt pour chaque entreprise
- optimiser et implémenter une étape de nettoyage des données entre docling et langchain
- développer un nouveau prompt pour d'autres attributs (effectifs totaux, proportion d'hommes et de femmes, etc)
- ajouter le titre du tableau ou des métadonnées extraites par ailleurs dans un second fichier de données pour le LLM